In [33]:
import pandas as pd

price_df = pd.read_parquet("../../data/Bloomberg Data Retrive.parquet")
# Filter the DataFrame by 'PLLFLNJK PLDP Index'
price_df = price_df[price_df['Name'] == 'PLLFLNJK PLDP Index']
# Drop the columns
price_df = price_df.drop(columns=['VOLUME', 'PX_ASK', 'PX_BID', 'PX_SETTLE', 'OPEN_INT', 'Name'])

In [34]:
price_df = price_df.iloc[0:3802]
price_df
# price_df.to_csv("../../data/Fine-Tuning/PLLFLNJK_PLDP_Index_Price.csv", index=False)

,Date,last_price
0,2009-02-02,7.000
1,2009-02-03,7.100
2,2009-02-04,7.000
3,2009-02-05,6.900
4,2009-02-06,6.950
...,...,...
3797,2024-03-26,9.668
3798,2024-03-27,9.523
3799,2024-03-28,9.509
3800,2024-04-01,9.494


In [35]:
price_df['previous price'] = price_df['last_price'].shift(1)
price_df['T+1 return'] = price_df['last_price'].pct_change()
price_df['T+3 return'] = price_df['last_price'].pct_change(3)
price_df['T+5 return'] = price_df['last_price'].pct_change(5)
price_df['T+10 return'] = price_df['last_price'].pct_change(10)
price_df['previous T+1 return'] = price_df['T+1 return'].shift(1)
price_df['previous T+3 return'] = price_df['T+3 return'].shift(1)
price_df['previous T+5 return'] = price_df['T+5 return'].shift(1)
price_df['previous T+10 return'] = price_df['T+10 return'].shift(1)
price_df['volatility_weekly'] = price_df['last_price'].rolling(window=5).std()
price_df['volatility_monthly'] = price_df['last_price'].rolling(window=21).std()
price_df['volatility_annual'] = price_df['last_price'].rolling(window=252).std()
price_df['previous volatility_weekly'] = price_df['volatility_weekly'].shift(1)
price_df['previous volatility_monthly'] = price_df['volatility_monthly'].shift(1)
price_df['previous volatility_annual'] = price_df['volatility_annual'].shift(1)
price_df


,Date,last_price,previous price,T+1 return,T+3 return,T+5 return,T+10 return,previous T+1 return,previous T+3 return,previous T+5 return,previous T+10 return,volatility_weekly,volatility_monthly,volatility_annual,previous volatility_weekly,previous volatility_monthly,previous volatility_annual
0,2009-02-02,7.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-02-03,7.100,7.000,0.014286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-02-04,7.000,7.100,-0.014085,NaN,NaN,NaN,0.014286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-02-05,6.900,7.000,-0.014286,-0.014286,NaN,NaN,-0.014085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-02-06,6.950,6.900,0.007246,-0.021127,NaN,NaN,-0.014286,-0.014286,NaN,NaN,0.074162,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3797,2024-03-26,9.668,9.831,-0.016580,0.024587,-0.031360,0.130364,0.070915,0.022784,-0.011463,0.116905,0.248253,0.590959,2.664385,0.316876,0.607568,2.660497
3798,2024-03-27,9.523,9.668,-0.014998,0.037364,-0.009259,0.092839,-0.016580,0.024587,-0.031360,0.130364,0.245459,0.569328,2.669041,0.248253,0.590959,2.664385
3799,2024-03-28,9.509,9.523,-0.001470,-0.032754,0.007736,0.081551,-0.014998,0.037364,-0.009259,0.092839,0.240775,0.547606,2.673772,0.245459,0.569328,2.669041
3800,2024-04-01,9.494,9.509,-0.001577,-0.017998,0.034205,0.038504,-0.001470,-0.032754,0.007736,0.081551,0.144314,0.513672,2.678252,0.240775,0.547606,2.673772


In [36]:
price_df
# price_df.to_csv("../../data/Fine-Tuning/PLLFLNJK_PLDP_FULL.csv", index=False)

,Date,last_price,previous price,T+1 return,T+3 return,T+5 return,T+10 return,previous T+1 return,previous T+3 return,previous T+5 return,previous T+10 return,volatility_weekly,volatility_monthly,volatility_annual,previous volatility_weekly,previous volatility_monthly,previous volatility_annual
0,2009-02-02,7.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-02-03,7.100,7.000,0.014286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-02-04,7.000,7.100,-0.014085,NaN,NaN,NaN,0.014286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-02-05,6.900,7.000,-0.014286,-0.014286,NaN,NaN,-0.014085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-02-06,6.950,6.900,0.007246,-0.021127,NaN,NaN,-0.014286,-0.014286,NaN,NaN,0.074162,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3797,2024-03-26,9.668,9.831,-0.016580,0.024587,-0.031360,0.130364,0.070915,0.022784,-0.011463,0.116905,0.248253,0.590959,2.664385,0.316876,0.607568,2.660497
3798,2024-03-27,9.523,9.668,-0.014998,0.037364,-0.009259,0.092839,-0.016580,0.024587,-0.031360,0.130364,0.245459,0.569328,2.669041,0.248253,0.590959,2.664385
3799,2024-03-28,9.509,9.523,-0.001470,-0.032754,0.007736,0.081551,-0.014998,0.037364,-0.009259,0.092839,0.240775,0.547606,2.673772,0.245459,0.569328,2.669041
3800,2024-04-01,9.494,9.509,-0.001577,-0.017998,0.034205,0.038504,-0.001470,-0.032754,0.007736,0.081551,0.144314,0.513672,2.678252,0.240775,0.547606,2.673772


In [37]:
news_path = '../../data/WSJ/WSJ_data_modified.csv'
news = pd.read_csv(news_path)
wsj_data_modified_reversed = news.iloc[::-1].reset_index(drop=True)

# data['Timestamp'] = pd.to_datetime(data['Timestamp']).dt.strftime('%Y-%m-%d')

In [38]:
from datetime import timedelta

# Convert 'Timestamp' column to datetime
wsj_data_modified_reversed['Timestamp'] = pd.to_datetime(wsj_data_modified_reversed['Timestamp'])

# Function to adjust date to the Friday of the same week if it's a weekend
def adjust_to_friday(date):
    if date.weekday() == 5:  # Saturday
        return date - timedelta(days=1)
    elif date.weekday() == 6:  # Sunday
        return date - timedelta(days=2)
    else:
        return date

# Apply the function to the 'Timestamp' column
wsj_data_modified_reversed['Timestamp'] = wsj_data_modified_reversed['Timestamp'].apply(adjust_to_friday)
wsj_data_modified_reversed


,Timestamp,Title,Theme,Summary
0,2020-04-17,Fall of Natural Gas Prices Speeds Energy Shift...,Gas Markets,When natural gas became as cheap as coal in th...
1,2020-04-20,Coronavirus Contract Disputes Start Hitting th...,U.S.,Companies are suspending or terminating busine...
2,2020-04-21,Shell Delays $1 Billion North Sea Development,Business,Royal Dutch Shell has postponed its Jackdaw na...
3,2020-04-27,Ship Orders Crash as Coronavirus Takes a Toll ...,Logistics Report,The economic fallout from coronavirus restrict...
4,2020-05-12,The Daily Shot: Tepid Signs of Stabilization i...,Dailyshot Blog,NaN
...,...,...,...,...
554,2024-03-14,Both Parties Go After Robert Hur's Report on C...,PODCAST,Special counsel Robert Hur received criticism ...
555,2024-03-29,The Climate Scientist Fossil-Fuel Companies Ca...,Gas Markets,Robert Howarth’s research on natural gas expor...
556,2024-03-29,Opinion: Green Shareholders Try to Sabotage Am...,Commentary,A wave of destructive proposals began after a ...
557,2024-04-02,Natural Gas Pioneer Charif Souki Ordered to Pa...,Pro Bankruptcy,The ruling stems from disputes with lenders wh...


In [39]:
# Convert the 'Date' columns in both dataframes to datetime for accurate merging
price_df['Date'] = pd.to_datetime(price_df['Date'])
wsj_data_modified_reversed['Timestamp'] = pd.to_datetime(wsj_data_modified_reversed['Timestamp'])

# Perform a left join of wsj_data_modified_reversed with pllflnjk_pldp_index_data based on the 'Timestamp'/'Date' columns
merged_data = wsj_data_modified_reversed.merge(price_df, left_on='Timestamp', right_on='Date', how='left')

# Since we're interested in adding columns from PLLFLNJK_PLDP_Index to WSJ_data_modified, we'll drop the 'Date' column from the result to avoid duplication
merged_data.drop('Date', axis=1, inplace=True)
merged_data


,Timestamp,Title,Theme,Summary,last_price,previous price,T+1 return,T+3 return,T+5 return,T+10 return,previous T+1 return,previous T+3 return,previous T+5 return,previous T+10 return,volatility_weekly,volatility_monthly,volatility_annual,previous volatility_weekly,previous volatility_monthly,previous volatility_annual
0,2020-04-17,Fall of Natural Gas Prices Speeds Energy Shift...,Gas Markets,When natural gas became as cheap as coal in th...,2.186,2.388,-0.084590,-0.069787,-0.069787,-0.034026,0.043706,0.016170,-0.005000,0.055236,0.079239,0.466530,1.103634,0.035961,0.492967,1.096186
1,2020-04-20,Coronavirus Contract Disputes Start Hitting th...,U.S.,Companies are suspending or terminating busine...,2.175,2.186,-0.005032,-0.049388,-0.074468,-0.059663,-0.084590,-0.069787,-0.069787,-0.034026,0.095466,0.428971,1.112447,0.079239,0.466530,1.103634
2,2020-04-21,Shell Delays $1 Billion North Sea Development,Business,Royal Dutch Shell has postponed its Jackdaw na...,2.040,2.175,-0.062069,-0.145729,-0.131915,-0.136691,-0.005032,-0.049388,-0.074468,-0.059663,0.130694,0.386660,1.123106,0.095466,0.428971,1.112447
3,2020-04-27,Ship Orders Crash as Coronavirus Takes a Toll ...,Logistics Report,The economic fallout from coronavirus restrict...,1.900,1.925,-0.012987,-0.050000,-0.126437,-0.191489,-0.006708,-0.056373,-0.119396,-0.180851,0.057678,0.222588,1.167786,0.100595,0.243722,1.156817
4,2020-05-12,The Daily Shot: Tepid Signs of Stabilization i...,Dailyshot Blog,NaN,2.438,2.325,0.048602,0.203951,0.308642,0.266494,0.058743,0.148148,0.247987,0.199690,0.182701,0.209373,1.238683,0.177756,0.206526,1.234485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,2024-03-14,Both Parties Go After Robert Hur's Report on C...,PODCAST,Special counsel Robert Hur received criticism ...,8.792,8.714,0.008951,-0.001136,0.016064,0.069847,0.018824,-0.048482,-0.029837,0.055091,0.221678,0.345803,2.633046,0.232003,0.339766,2.628066
555,2024-03-29,The Climate Scientist Fossil-Fuel Companies Ca...,Gas Markets,Robert Howarth’s research on natural gas expor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
556,2024-03-29,Opinion: Green Shareholders Try to Sabotage Am...,Commentary,A wave of destructive proposals began after a ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557,2024-04-02,Natural Gas Pioneer Charif Souki Ordered to Pa...,Pro Bankruptcy,The ruling stems from disputes with lenders wh...,9.383,9.494,-0.011692,-0.014701,-0.045570,-0.056511,-0.001577,-0.017998,0.034205,0.038504,0.101702,0.477583,2.683359,0.144314,0.513672,2.678252


In [42]:
# merged_data.to_csv("../../data/Fine-Tuning/merged_raw.csv", index=False)
cleaned_df = merged_data.dropna(subset=['last_price'])
# cleaned_df = cleaned_df[cleaned_df['return'] != 0]
cleaned_df = cleaned_df[cleaned_df['Theme'] != 'PODCAST']
cleaned_df = cleaned_df[cleaned_df['Theme'] != 'Dailyshot Blog']
cleaned_df
# cleaned_df.to_csv("../../data/Fine-Tuning/merged_cleaned.csv", index=False)

,Timestamp,Title,Theme,Summary,last_price,previous price,T+1 return,T+3 return,T+5 return,T+10 return,previous T+1 return,previous T+3 return,previous T+5 return,previous T+10 return,volatility_weekly,volatility_monthly,volatility_annual,previous volatility_weekly,previous volatility_monthly,previous volatility_annual
0,2020-04-17,Fall of Natural Gas Prices Speeds Energy Shift...,Gas Markets,When natural gas became as cheap as coal in th...,2.186,2.388,-0.084590,-0.069787,-0.069787,-0.034026,0.043706,0.016170,-0.005000,0.055236,0.079239,0.466530,1.103634,0.035961,0.492967,1.096186
1,2020-04-20,Coronavirus Contract Disputes Start Hitting th...,U.S.,Companies are suspending or terminating busine...,2.175,2.186,-0.005032,-0.049388,-0.074468,-0.059663,-0.084590,-0.069787,-0.069787,-0.034026,0.095466,0.428971,1.112447,0.079239,0.466530,1.103634
2,2020-04-21,Shell Delays $1 Billion North Sea Development,Business,Royal Dutch Shell has postponed its Jackdaw na...,2.040,2.175,-0.062069,-0.145729,-0.131915,-0.136691,-0.005032,-0.049388,-0.074468,-0.059663,0.130694,0.386660,1.123106,0.095466,0.428971,1.112447
3,2020-04-27,Ship Orders Crash as Coronavirus Takes a Toll ...,Logistics Report,The economic fallout from coronavirus restrict...,1.900,1.925,-0.012987,-0.050000,-0.126437,-0.191489,-0.006708,-0.056373,-0.119396,-0.180851,0.057678,0.222588,1.167786,0.100595,0.243722,1.156817
6,2020-05-21,A Natural Gas Exec Turned His Colorado Ranch I...,Homes,Egypt-born entrepreneur Charif Souki invested ...,2.125,2.173,-0.022089,-0.110134,-0.113845,0.049383,-0.062149,-0.080406,-0.113061,0.073086,0.117436,0.220398,1.276431,0.092020,0.220789,1.269066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,2024-02-23,Natural Gas Hasn’t Been This Cheap in Decades,Gas Markets,The lowest inflation-adjusted prices in at lea...,7.983,8.230,-0.030012,-0.037149,-0.082203,-0.155953,-0.006039,-0.031537,-0.077871,-0.120351,0.184099,0.553418,2.518889,0.195951,0.502700,2.506196
551,2024-03-11,Elliott Bets $500 Million on Gas Producer Back...,Pro PE Fund News,Paul Singer’s Elliott Investment is plowing mo...,8.802,9.158,-0.038873,-0.020040,0.063557,0.102869,0.058361,0.021756,0.099796,0.147188,0.191831,0.374692,2.617132,0.347792,0.422480,2.608159
552,2024-03-13,Biggest Gains in Financial Strength in the Man...,Journal Reports: Leadership,"Royal Caribbean heads the list, followed by Ch...",8.714,8.553,0.018824,-0.048482,-0.029837,0.055091,-0.028289,-0.011557,-0.045744,0.050221,0.232003,0.339766,2.628066,0.244709,0.346011,2.624210
553,2024-03-14,Shell to Slow Pace of Carbon-Emission Cuts,Sustainable Business,Anglo-Dutch energy heavyweight said it now tar...,8.792,8.714,0.008951,-0.001136,0.016064,0.069847,0.018824,-0.048482,-0.029837,0.055091,0.221678,0.345803,2.633046,0.232003,0.339766,2.628066


In [43]:
# Grouping by Timestamp and merging the Title, Theme, and Summary columns as specified
# If there's only one record for a day, it will be left as is, otherwise they will be merged with numbering

# Define a custom function to merge columns with numbering if there are multiple records, else leave as is
def merge_with_numbering(series):
    if len(series) == 1:
        return series.iloc[0]
    else:
        return ''.join(f"({i+1}) {x} " for i, x in enumerate(series))

# Group by 'Timestamp' and apply the custom function to 'Title', 'Theme', and 'Summary'
grouped = cleaned_df.groupby('Timestamp').agg({
    'Title': merge_with_numbering,
    'Theme': merge_with_numbering,
    'Summary': merge_with_numbering,
    'last_price': 'first',  # Keeping the rest of the columns the same (assuming they're identical for grouped records)
    'previous price': 'first',
    'T+1 return': 'first',
    'T+3 return': 'first',
    'T+5 return': 'first',
    'T+10 return': 'first',
    'previous T+1 return': 'first',
    'previous T+3 return': 'first',
    'previous T+5 return': 'first',
    'previous T+10 return': 'first',
    'volatility_weekly': 'first',
    'volatility_monthly': 'first',
    'volatility_annual': 'first',
    'previous volatility_weekly': 'first',
    'previous volatility_monthly': 'first',
    'previous volatility_annual': 'first'
}).reset_index()

# Display the first few rows to verify the transformation
grouped

,Timestamp,Title,Theme,Summary,last_price,previous price,T+1 return,T+3 return,T+5 return,T+10 return,previous T+1 return,previous T+3 return,previous T+5 return,previous T+10 return,volatility_weekly,volatility_monthly,volatility_annual,previous volatility_weekly,previous volatility_monthly,previous volatility_annual
0,2020-04-17,Fall of Natural Gas Prices Speeds Energy Shift...,Gas Markets,When natural gas became as cheap as coal in th...,2.186,2.388,-0.084590,-0.069787,-0.069787,-0.034026,0.043706,0.016170,-0.005000,0.055236,0.079239,0.466530,1.103634,0.035961,0.492967,1.096186
1,2020-04-20,Coronavirus Contract Disputes Start Hitting th...,U.S.,Companies are suspending or terminating busine...,2.175,2.186,-0.005032,-0.049388,-0.074468,-0.059663,-0.084590,-0.069787,-0.069787,-0.034026,0.095466,0.428971,1.112447,0.079239,0.466530,1.103634
2,2020-04-21,Shell Delays $1 Billion North Sea Development,Business,Royal Dutch Shell has postponed its Jackdaw na...,2.040,2.175,-0.062069,-0.145729,-0.131915,-0.136691,-0.005032,-0.049388,-0.074468,-0.059663,0.130694,0.386660,1.123106,0.095466,0.428971,1.112447
3,2020-04-27,Ship Orders Crash as Coronavirus Takes a Toll ...,Logistics Report,The economic fallout from coronavirus restrict...,1.900,1.925,-0.012987,-0.050000,-0.126437,-0.191489,-0.006708,-0.056373,-0.119396,-0.180851,0.057678,0.222588,1.167786,0.100595,0.243722,1.156817
4,2020-05-21,A Natural Gas Exec Turned His Colorado Ranch I...,Homes,Egypt-born entrepreneur Charif Souki invested ...,2.125,2.173,-0.022089,-0.110134,-0.113845,0.049383,-0.062149,-0.080406,-0.113061,0.073086,0.117436,0.220398,1.276431,0.092020,0.220789,1.269066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,2024-02-23,(1) Corrections & Amplifications (2) Natural G...,(1) Corrections (2) Gas Markets,(1) Corrections & Amplifications for the editi...,7.983,8.230,-0.030012,-0.037149,-0.082203,-0.155953,-0.006039,-0.031537,-0.077871,-0.120351,0.184099,0.553418,2.518889,0.195951,0.502700,2.506196
322,2024-03-11,Elliott Bets $500 Million on Gas Producer Back...,Pro PE Fund News,Paul Singer’s Elliott Investment is plowing mo...,8.802,9.158,-0.038873,-0.020040,0.063557,0.102869,0.058361,0.021756,0.099796,0.147188,0.191831,0.374692,2.617132,0.347792,0.422480,2.608159
323,2024-03-13,Biggest Gains in Financial Strength in the Man...,Journal Reports: Leadership,"Royal Caribbean heads the list, followed by Ch...",8.714,8.553,0.018824,-0.048482,-0.029837,0.055091,-0.028289,-0.011557,-0.045744,0.050221,0.232003,0.339766,2.628066,0.244709,0.346011,2.624210
324,2024-03-14,Shell to Slow Pace of Carbon-Emission Cuts,Sustainable Business,Anglo-Dutch energy heavyweight said it now tar...,8.792,8.714,0.008951,-0.001136,0.016064,0.069847,0.018824,-0.048482,-0.029837,0.055091,0.221678,0.345803,2.633046,0.232003,0.339766,2.628066


In [44]:
grouped.to_csv("../../data/Fine-Tuning/Pretraining_Raw.csv", index=False)